# Data Preparation for Multi-Asset Universe

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime

In [2]:
start_equity = "2005-01-01"
end_all = "2025-06-01"
start_crypto = "2015-01-01"
start_unified = "2007-01-01"

## Assets

In [3]:
us_equities = [
    "AAPL", "MSFT", "AMZN", "GOOGL", "META",
    "JNJ", "V", "JPM", "PG", "DIS",
    "NVDA", "HD", "BAC", "MA", "PFE",
    "VZ", "INTC", "CSCO", "KO", "PEP",
    "MRK", "WMT", "CVX", "ADBE", "T"
]

intl_equities = [
    "SAP.DE", "BMW.DE", "DTG.DE", "BAS.DE", "AIR.PA",   # Germany/France
    "SHEL.L", "VOD.L", "HSBA.L", "AZN.L", "BP.L",      # UK
    "7203.T", "6861.T", "9984.T", "6954.T", "8411.T",  # Japan
    "CS.PA", "SAN.PA", "BNP.PA", "MC.PA", "OR.PA",     # France
    "ULVR.L", "BARC.L", "RIO.L", "GLEN.L", "DLG.L"     # UK
]

fx_map = {
    **{ticker: "EURUSD=X" for ticker in ["SAP.DE", "BMW.DE", "DTG.DE", "BAS.DE", "AIR.PA",
                                         "CS.PA", "SAN.PA", "BNP.PA", "MC.PA", "OR.PA"]},
    **{ticker: "GBPUSD=X" for ticker in ["SHEL.L", "VOD.L", "HSBA.L", "AZN.L", "BP.L",
                                         "ULVR.L", "BARC.L", "RIO.L", "GLEN.L", "DLG.L"]},
    **{ticker: "JPYUSD=X" for ticker in ["7203.T", "6861.T", "9984.T", "6954.T", "8411.T"]}
}

us_bond_etfs = ["TLT", "IEF", "SHY", "VGSH", "BND"]  # example U.S. Treasury ETFs
intl_bond_etfs = ["BWX", "PHAU.L", "2561.T"]
fx_map.update({
    "BUNL.DE": "EURUSD=X",
    "JGBB.DE": "JPYUSD=X",
    "GLDB.L": "GBPUSD=X",
    "XG7T.DE": "EURUSD=X"
})

commodity_etfs = ["GLD", "USO", "CPER", "DBA", "XLE"]

crypto_tickers = ["BTC-USD", "ETH-USD", "SOL-USD"]

## Helper Methods

In [4]:
def _get_price_series(df: pd.DataFrame) -> pd.DataFrame:
    """
    Given a DataFrame from yf.download, return the 'Close' prices.
    If df.columns is a MultiIndex with a 'Close' level, extract that;
    otherwise assume df itself contains the desired series.
    """
    if isinstance(df.columns, pd.MultiIndex):
        if "Close" in df.columns.get_level_values(0):
            return df["Close"]
        else:
            return df
    else:
        return df


def download_and_process_equities(tickers, start_date, end_date, missing_thresh=0.05):
    """
    Download Close prices for tickers from Yahoo Finance,
    forward-fill single-day gaps, drop series with >missing_thresh missing,
    compute log-returns, and return DataFrame of returns.
    """
    raw = yf.download(tickers, start=start_date, end=end_date, progress=False)
    prices = _get_price_series(raw).ffill(limit=1)

    missing_pct = prices.isna().mean()
    keep = missing_pct[missing_pct <= missing_thresh].index.tolist()
    data = prices[keep].dropna(how="all")

    returns = np.log(data / data.shift(1)).dropna(how="all")
    return returns


def download_and_process_intl_equities(tickers, start_date, end_date, fx_map, missing_thresh=0.05):
    """
    Download local-currency Close prices for tickers, forward-fill, drop if >missing_thresh missing,
    compute local log-returns, convert to USD returns via FX log-returns, and return DataFrame.
    """
    raw_loc = yf.download(tickers, start=start_date, end=end_date, progress=False)
    prices_loc = _get_price_series(raw_loc).ffill(limit=1)

    returns_usd = {}
    for ticker in tickers:
        if ticker not in prices_loc.columns:
            continue
        s_loc = prices_loc[ticker].copy()
        if s_loc.isna().mean() > missing_thresh:
            continue
        s_loc = s_loc.ffill()
        r_loc = np.log(s_loc / s_loc.shift(1))

        fx_pair = fx_map.get(ticker)
        if fx_pair is None:
            continue

        raw_fx = yf.download(fx_pair, start=start_date, end=end_date, progress=False)
        prices_fx_df = _get_price_series(raw_fx).ffill(limit=1)
        if isinstance(prices_fx_df, pd.DataFrame):
            fx_series = prices_fx_df[fx_pair] if fx_pair in prices_fx_df.columns else prices_fx_df.iloc[:, 0]
        else:
            fx_series = prices_fx_df

        if fx_series.isna().mean() > missing_thresh:
            continue
        if fx_pair == "JPYUSD=X":
            fx_series = 1.0 / fx_series
        fx_series = fx_series.ffill()
        r_fx = np.log(fx_series / fx_series.shift(1))

        df = pd.concat([r_loc, r_fx], axis=1).dropna(how="any")
        returns_usd[ticker] = df.iloc[:, 0] + df.iloc[:, 1]

    return pd.DataFrame(returns_usd).sort_index()


def download_and_process_bonds(tickers, start_date, end_date, fx_map, missing_thresh=0.05):
    """
    Download bond ETF Close prices, forward-fill, drop if >missing_thresh missing,
    compute local log-returns, convert to USD via FX if needed, and return DataFrame.
    """
    raw = yf.download(tickers, start=start_date, end=end_date, progress=False)
    prices = _get_price_series(raw).ffill(limit=1)

    returns_usd = {}
    for ticker in tickers:
        if ticker not in prices.columns:
            continue
        s = prices[ticker].copy()
        if s.isna().mean() > missing_thresh:
            continue
        s = s.ffill()
        r_local = np.log(s / s.shift(1))

        fx_pair = fx_map.get(ticker)
        if fx_pair is None:
            returns_usd[ticker] = r_local
            continue

        raw_fx = yf.download(fx_pair, start=start_date, end=end_date, progress=False)
        prices_fx_df = _get_price_series(raw_fx).ffill(limit=1)
        if isinstance(prices_fx_df, pd.DataFrame):
            fx_series = prices_fx_df[fx_pair] if fx_pair in prices_fx_df.columns else prices_fx_df.iloc[:, 0]
        else:
            fx_series = prices_fx_df

        if fx_series.isna().mean() > missing_thresh:
            continue
        if fx_pair == "JPYUSD=X":
            fx_series = 1.0 / fx_series
        fx_series = fx_series.ffill()
        r_fx = np.log(fx_series / fx_series.shift(1))

        df = pd.concat([r_local, r_fx], axis=1).dropna(how="any")
        returns_usd[ticker] = df.iloc[:, 0] + df.iloc[:, 1]

    return pd.DataFrame(returns_usd).sort_index()


def download_and_process_commodities(tickers, start_date, end_date, missing_thresh=0.05):
    """
    Download commodity ETF Close prices (USD), forward-fill, drop if >missing_thresh missing,
    compute log-returns, and return DataFrame.
    """
    raw = yf.download(tickers, start=start_date, end=end_date, progress=False)
    prices = _get_price_series(raw).ffill(limit=1)

    missing_pct = prices.isna().mean()
    keep = missing_pct[missing_pct <= missing_thresh].index.tolist()
    data = prices[keep].dropna(how="all")

    returns = np.log(data / data.shift(1)).dropna(how="all")
    return returns


def download_and_process_crypto(tickers, start_date, end_date, missing_thresh=0.02):
    """
    Download crypto USD Close prices, forward-fill, drop if >missing_thresh missing,
    compute log-returns, and return DataFrame.
    """
    raw = yf.download(tickers, start=start_date, end=end_date, progress=False)
    prices = _get_price_series(raw).ffill(limit=1)

    missing_pct = prices.isna().mean()
    keep = missing_pct[missing_pct <= missing_thresh].index.tolist()
    data = prices[keep].dropna(how="all")

    returns = np.log(data / data.shift(1)).dropna(how="all")
    return returns


## Load data

In [5]:
# US Equities (2005–2025)
us_eq_rets = download_and_process_equities(
    us_equities, start_equity, end_all, missing_thresh=0.05
)

# International Equities (2005–2025)
intl_eq_rets = download_and_process_intl_equities(
    intl_equities, start_equity, end_all, fx_map, missing_thresh=0.05
)

# US Bond ETFs (2005–2025)
us_bond_rets = download_and_process_bonds(
    us_bond_etfs, start_equity, end_all, fx_map={}, missing_thresh=0.05
)

# International Bond ETFs (2007–2025)
intl_bond_rets = download_and_process_bonds(
    intl_bond_etfs, "2007-01-01", end_all, fx_map, missing_thresh=0.05
)

# Commodities (2005–2025 for GLD, USO, CPER; 2007–2025 for DBA, XLE)
# We'll download all from 2005 but drop missing >5%
comm_rets = download_and_process_commodities(
    commodity_etfs, start_equity, end_all, missing_thresh=0.05
)

# Cryptocurrencies (2015–2025)
crypto_rets = download_and_process_crypto(
    crypto_tickers, start_crypto, end_all, missing_thresh=0.02
)

YF.download() has changed argument auto_adjust default to True


# Align & Merge into Unified Dataset

In [6]:
# 6.1 Truncate each DataFrame to start_unified (2007-01-01) if earlier
us_eq_rets = us_eq_rets[us_eq_rets.index >= start_unified]
intl_eq_rets = intl_eq_rets[intl_eq_rets.index >= start_unified]
us_bond_rets = us_bond_rets[us_bond_rets.index >= start_unified]
intl_bond_rets = intl_bond_rets[intl_bond_rets.index >= start_unified]
comm_rets = comm_rets[comm_rets.index >= start_unified]
crypto_rets = crypto_rets[crypto_rets.index >= start_unified]

# 6.2 Intersect dates across all asset‐class return DataFrames
all_dates = us_eq_rets.index
for df in [intl_eq_rets, us_bond_rets, intl_bond_rets, comm_rets, crypto_rets]:
    all_dates = all_dates.intersection(df.index)

# 6.3 Reindex each DataFrame to all_dates, forward‐fill remaining gaps for non‐US where needed
def reindex_and_ffill(df, dates):
    return df.reindex(dates).ffill()

us_eq_rets = reindex_and_ffill(us_eq_rets, all_dates)
intl_eq_rets = reindex_and_ffill(intl_eq_rets, all_dates)
us_bond_rets = reindex_and_ffill(us_bond_rets, all_dates)
intl_bond_rets = reindex_and_ffill(intl_bond_rets, all_dates)
comm_rets = reindex_and_ffill(comm_rets, all_dates)
crypto_rets = reindex_and_ffill(crypto_rets, all_dates)

# 6.4 Concatenate along columns to get final T x N matrix
all_returns = pd.concat(
    [us_eq_rets, intl_eq_rets, us_bond_rets, intl_bond_rets, comm_rets, crypto_rets],
    axis=1
).sort_index()

# 6.5 Final missing‐value check (should be none if forward‐filled)
assert all_returns.isna().sum().sum() == 0, "There are still missing values!"

In [10]:
all_returns.head()

,AAPL,ADBE,AMZN,BAC,CSCO,CVX,DIS,GOOGL,HD,INTC,...,BARC.L,RIO.L,TLT,IEF,SHY,BWX,PHAU.L,GLD,XLE,BTC-USD
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,-0.009558,-0.004964,-0.005914,0.000559,-0.000724,0.003560,-0.004682,-0.002094,-0.014780,0.001927,...,-0.001516,-0.010128,0.011057,0.005082,0.001066,-0.004347,-0.006472,0.004393,0.004663,0.002489
2015-01-05,-0.028576,-0.004989,-0.020731,-0.029481,-0.020122,-0.040792,-0.014721,-0.019238,-0.021204,-0.011340,...,-0.054474,-0.048661,0.015587,0.006083,0.000000,-0.002180,0.004147,0.014965,-0.042248,0.038169
2015-01-06,0.000094,-0.020350,-0.023098,-0.030376,-0.000370,-0.000463,-0.005318,-0.024989,-0.003066,-0.018813,...,-0.020449,0.019193,0.017857,0.006695,0.000473,0.003088,0.011913,0.011334,-0.014800,0.041796
2015-01-07,0.013925,0.008190,0.010544,0.004734,0.009200,-0.000833,0.010178,-0.002945,0.033700,0.020758,...,-0.006834,-0.001823,-0.001977,-0.000185,0.000473,-0.002724,0.002128,-0.005909,0.002128,0.028073
2015-01-08,0.037703,0.025135,0.006813,0.020451,0.007663,0.022625,0.010288,0.003478,0.021883,0.018430,...,0.025575,0.019692,-0.013331,-0.004087,-0.000118,-0.001456,0.002038,-0.004217,0.022201,-0.038046


In [8]:
all_returns.to_csv("returns", sep=",", index=True)